In [1]:
! pip install groq
! pip install datasets

In [3]:
# import os
# from groq import Groq

# client = Groq(
#     api_key="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",
# )

# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "Explain the importance of fast language models",
#         }
#     ],
#     model="llama3-8b-8192",
# )

# print(chat_completion.choices[0].message.content),

Fast language models have become increasingly important in natural language processing (NLP) and machine learning in recent years, due to their ability to process and respond to large amounts of text quickly and accurately. Here are some reasons why fast language models are crucial:

1. **Scalability:** Fast language models can handle large volumes of text data, making them ideal for applications where scalability is essential, such as chatbots, customer service, and social media analysis.

2. **Real-time processing:** Fast language models enable real-time processing and response, allowing for applications like language translation, text summarization, and sentiment analysis to be performed instantaneously.

3. **Improved user experience:** Fast language models can respond quickly to user queries, providing a seamless and interactive experience for users. For example, a language translation model can instantly translate text for real-time communication.

4. **Handling unstructured data

(None,)

Self Correction for Gemma 7B Model

In [1]:
import os
from datasets import load_dataset
from groq import Groq

# Initialize Groq client
client = Groq(
    api_key="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"  # Ensure you have the Groq API Key set as an environment variable
)

# Load the SVAMP dataset
dataset = load_dataset("ChilleD/SVAMP", split='test')

# Define the model for Groq API calls
model = "llama-3.1-8b-instant"  # Change to the model you want to use

# Prompt templates
prompt_1 = "Can you solve the following math problem? {problem} Explain your reasoning. Your final answer should be a single numerical number, in the form \\boxed{{answer}}, at the end of your response. Don't generate any additional text inside the \\boxed{{answer}}"
prompt_2 = "Review your previous answer and find problems with your answer."
prompt_3 = "Based on the problems you found, improve your answer. Please reiterate your answer, with your final answer a single numerical number, in the form \\boxed{{answer}}. Don't generate any additional text inside the \\boxed{{answer}}"

# Function to make Groq API call
def get_groq_response(chat_history):
    chat_completion = client.chat.completions.create(
        messages=chat_history,
        model=model
    )
    return chat_completion.choices[0].message.content

# Evaluate self-correction on a sample of the dataset
def evaluate_self_correction(problem):
    # Prompt 1: Solve the problem
    
    chat_history = [{"role": "user", "content": prompt_1.format(problem=problem)}]

    response_1 = get_groq_response(chat_history)
    # print(f"Response 1: {response_1}")

    chat_history.append({"role": "assistant", "content": response_1})
    chat_history.append({"role": "user", "content": prompt_2})

    # Prompt 2: Review the previous answer
    response_2 = get_groq_response(chat_history)
    # print(f"Response 2: {response_2}")

    chat_history.append({"role": "assistant", "content": response_2})
    chat_history.append({"role": "user", "content": prompt_3})

    # Prompt 3: Improve the answer based on the review
    response_3 = get_groq_response(chat_history)
    # print(f"Response 3: {response_3}")

    return response_1, response_2, response_3

doubtful_responses = []

def extract_answer(response):

    start_index = response.find("\\boxed{") + len("\\boxed{")

    end_index = response.find("}", start_index)

    answer = response[start_index:end_index]

    # if(end_index == -1):
    #     end_index = response.find("]", start_index)
    # if(end_index == -1):
    #     end_index = response.find(")", start_index)

    # print("Response: ", response)

    # print("Start: ", start_index)
    # print("End: ", end_index)

    # print("Value: "+response[start_index:end_index])

    # answer = ""
    # flag = False
    # for i in range(start_index, end_index):
    #     if response[i].isdigit():
    #         flag = True
    #         answer += response[i]
    #     else:
    #         if flag:
    #             flag=False
    #             break

    return int(answer)

# Evaluate on the first 5 problems in the dataset

correct_1 = 0
correct_2 = 0

for idx in range(len(dataset)):
    print("Idx: ", idx)
    problem = dataset[idx]['Body'] + " " + dataset[idx]['Question']
    correct_answer = int(dataset[idx]['Answer'])
    # print(f"\nProblem {idx+1}: {problem}")
    response_1, response_2, response_3 = evaluate_self_correction(problem)

    initial_answer = 0
    final_answer = 0

    try:
        initial_answer = extract_answer(response_1)
    except:
        doubtful_responses.append(response_1)
        continue
    # try:
    #     answer_1 = int(answer_1.split(" ")[0])
    # except: 
    #     print("Response 1: "+ response_1)
    #     print("Answer 1: "+ answer_1)
    #     break
    # answer_2 = extract_answer(response_2)
    try:
        final_answer = extract_answer(response_3)
    except: 
        doubtful_responses.append(response_3)
        continue
    # try:
    #     answer_2 = int(answer_2.split(" ")[0])
    # except: 
    #     print("Response 2: "+ response_3)
    #     print("Answer 2: "+ answer_2)
    #     break
    # print("Initial Answer: ", initial_answer)
    # print("Final Answer: ", final_answer)
    
    if(initial_answer == correct_answer):
        correct_1 += 1
    if(final_answer == correct_answer):
        correct_2 += 1


accuracy_1 = correct_1/(100-len(doubtful_responses)) * 100
accuracy_2 = correct_2/(100-len(doubtful_responses)) * 100

print("Accuracy before self-correction: ", correct_1)
print("Accuracy after self-correction: ", correct_2)

print("Incorrect Format: ", len(doubtful_responses))

    






/Users/aayush/miniforge3/envs/llm_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Idx:  0
Idx:  1
Idx:  2
Idx:  3
Idx:  4
Idx:  5
Idx:  6
Idx:  7
Idx:  8
Idx:  9
Idx:  10
Idx:  11
Idx:  12
Idx:  13
Idx:  14
Idx:  15
Idx:  16
Idx:  17
Idx:  18
Idx:  19
Idx:  20
Idx:  21
Idx:  22
Idx:  23
Idx:  24
Idx:  25
Idx:  26
Idx:  27
Idx:  28
Idx:  29
Idx:  30
Idx:  31
Idx:  32
Idx:  33
Idx:  34
Idx:  35
Idx:  36
Idx:  37
Idx:  38
Idx:  39
Idx:  40
Idx:  41
Idx:  42
Idx:  43
Idx:  44
Idx:  45
Idx:  46
Idx:  47
Idx:  48
Idx:  49
Idx:  50
Idx:  51
Idx:  52
Idx:  53
Idx:  54
Idx:  55
Idx:  56
Idx:  57
Idx:  58
Idx:  59
Idx:  60
Idx:  61
Idx:  62
Idx:  63
Idx:  64
Idx:  65
Idx:  66
Idx:  67
Idx:  68
Idx:  69
Idx:  70
Idx:  71
Idx:  72
Idx:  73
Idx:  74
Idx:  75
Idx:  76
Idx:  77
Idx:  78
Idx:  79
Idx:  80
Idx:  81
Idx:  82
Idx:  83
Idx:  84
Idx:  85
Idx:  86
Idx:  87
Idx:  88
Idx:  89
Idx:  90
Idx:  91
Idx:  92
Idx:  93
Idx:  94
Idx:  95
Idx:  96
Idx:  97
Idx:  98
Idx:  99
Idx:  100
Idx:  101
Idx:  102
Idx:  103
Idx:  104
Idx:  105
Idx:  106
Idx:  107
Idx:  108
Idx:  109
Idx:  110


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01j8wk53dnejab3ymjjsy4vnwk` on : Limit 500000, Used 499943, Requested 914. Please try again in 2m27.9284s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [4]:
import os
from datasets import load_dataset
from groq import Groq

# Initialize Groq client
client = Groq(
    api_key="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"  # Ensure you have the Groq API Key set as an environment variable
)

# Load the SVAMP dataset
dataset = load_dataset("ChilleD/SVAMP", split='test')

# Define the model for Groq API calls
model = "llama-3.1-8b-instant"  # Change to the model you want to use

# Prompt templates
prompt_1 = "Can you solve the following math problem? {problem} Explain your reasoning. Your final answer should be a single numerical number, in the form \\boxed{{answer}}, at the end of your response. Don't generate any additional text inside the \\boxed{{answer}}"
prompt_2 = "Review your previous answer and find problems with your answer."
prompt_3 = "Based on the problems you found, improve your answer. Please reiterate your answer, with your final answer a single numerical number, in the form \\boxed{{answer}}. Don't generate any additional text inside the \\boxed{{answer}}"

extraction_prompt = "Please extract the answer from the above text. Give the answer in the form \\boxed{{answer}} where {answer} is a single numerical number."

# Function to make Groq API call
def get_groq_response(chat_history):
    chat_completion = client.chat.completions.create(
        messages=chat_history,
        model=model,
        temperature=0
    )
    return chat_completion.choices[0].message.content

# Evaluate self-correction on a sample of the dataset
def evaluate_self_correction(problem):
    # Prompt 1: Solve the problem
    
    chat_history = [{"role": "user", "content": prompt_1.format(problem=problem)}]

    response_1 = get_groq_response(chat_history)
    # print(f"Response 1: {response_1}")

    initial_answer = get_groq_response([{"role": "user", "content": response_1+"\n"+extraction_prompt}])

    chat_history.append({"role": "assistant", "content": response_1})
    chat_history.append({"role": "user", "content": prompt_2})

    # Prompt 2: Review the previous answer
    response_2 = get_groq_response(chat_history)
    # print(f"Response 2: {response_2}")

    chat_history.append({"role": "assistant", "content": response_2})
    chat_history.append({"role": "user", "content": prompt_3})

    # Prompt 3: Improve the answer based on the review
    response_3 = get_groq_response(chat_history)

    final_answer = get_groq_response([{"role": "user", "content": response_3+"\n"+extraction_prompt}])
    # print(f"Response 3: {response_3}")

    return response_1, response_2, response_3, initial_answer, final_answer

doubtful_responses = []

def extract_answer(response):

    start_index = response.find("\\boxed{") + len("\\boxed{")

    end_index = response.find("}", start_index)

    answer = response[start_index:end_index]

    # if(end_index == -1):
    #     end_index = response.find("]", start_index)
    # if(end_index == -1):
    #     end_index = response.find(")", start_index)

    # print("Response: ", response)

    # print("Start: ", start_index)
    # print("End: ", end_index)

    # print("Value: "+response[start_index:end_index])

    # answer = ""
    # flag = False
    # for i in range(start_index, end_index):
    #     if response[i].isdigit():
    #         flag = True
    #         answer += response[i]
    #     else:
    #         if flag:
    #             flag=False
    #             break

    return int(answer)

# Evaluate on the first 5 problems in the dataset

correct_1 = 0
correct_2 = 0

for idx in range(5):
    print("Idx: ", idx)
    problem = dataset[idx]['Body'] + " " + dataset[idx]['Question']
    correct_answer = int(dataset[idx]['Answer'])
    # print(f"\nProblem {idx+1}: {problem}")
    response_1, response_2, response_3, initial_answer, final_answer = evaluate_self_correction(problem)

    # print("Response 1: ", response_1)
    # print("Response 2: ", response_2)
    # print("Response 3: ", response_3)

    try:
        initial_answer = extract_answer(initial_answer)
    except:
        doubtful_responses.append(response_1)
        continue
    # try:
    #     answer_1 = int(answer_1.split(" ")[0])
    # except: 
    #     print("Response 1: "+ response_1)
    #     print("Answer 1: "+ answer_1)
    #     break
    # answer_2 = extract_answer(response_2)
    try:
        final_answer = extract_answer(final_answer)
    except: 
        doubtful_responses.append(response_3)
        continue
    # try:
    #     answer_2 = int(answer_2.split(" ")[0])
    # except: 
    #     print("Response 2: "+ response_3)
    #     print("Answer 2: "+ answer_2)
    #     break
    # print("Initial Answer: ", initial_answer)
    # print("Final Answer: ", final_answer)
    
    if(initial_answer == correct_answer):
        correct_1 += 1
    if(final_answer == correct_answer):
        correct_2 += 1


accuracy_1 = correct_1/(5-len(doubtful_responses)) * 100
accuracy_2 = correct_2/(5-len(doubtful_responses)) * 100

print("Accuracy before self-correction: ", accuracy_1)
print("Accuracy after self-correction: ", accuracy_2)

print("Incorrect Format: ", len(doubtful_responses))

    






Idx:  0
Response 1:  To find out how many more bird families flew away to Africa than those that flew away to Asia, we need to compare the number of bird families that went to Africa and the number that went to Asia. 

The number of bird families that went to Asia is 35.
The number of bird families that went to Africa is 62.

To find the difference between the two, we subtract the number of bird families that went to Asia from the number that went to Africa. 
62 - 35 = 27

So, 27 more bird families flew away to Africa than those that flew away to Asia.
\boxed{27}
Response 2:  Upon reviewing my previous answer, I realized that I correctly solved the problem, but I need to consider the initial question's wording. It asks how many more bird families flew away to Africa than those that flew away to Asia. 

By subtracting the number of bird families that went to Asia from the number that went to Africa, I provided the correct answer, which is 27. However, this answer doesn't directly imply 

In [2]:
len(doubtful_responses)

154

In [25]:
print("3319921".isdigit())

True


In [11]:
a = '145'
b = int(a)
print(b)

145


LL

In [5]:
import os
from datasets import load_dataset
from groq import Groq

# Initialize Groq client
client = Groq(
    api_key="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"  # Ensure you have the Groq API Key set as an environment variable
)

# Load the SVAMP dataset
dataset = load_dataset("ChilleD/SVAMP", split='test')

# Define the model for Groq API calls
model = "llama-3.1-8b-instant"  # Change to the model you want to use

# Prompt templates
prompt_1 = "Can you solve the following math problem? {problem} Explain your reasoning. Your final answer should be a single numerical number, in the form \\boxed{{answer}}, at the end of your response. Don't generate any additional text inside the \\boxed{{answer}}"
prompt_2 = "Review your previous answer and find problems with your answer."
prompt_3 = "Based on the problems you found, improve your answer. Please reiterate your answer, with your final answer a single numerical number, in the form \\boxed{{answer}}. Don't generate any additional text inside the \\boxed{{answer}}"

extraction_prompt = "Please extract the answer from the above text. Give the answer in the form \\boxed{{answer}} where {answer} is a single numerical number."

# Function to make Groq API call
def get_groq_response(chat_history):
    chat_completion = client.chat.completions.create(
        messages=chat_history,
        model=model,
        temperature=0
    )
    return chat_completion.choices[0].message.content

# Evaluate self-correction on a sample of the dataset
def evaluate_self_correction(problem):
    # Prompt 1: Solve the problem
    
    chat_history = [{"role": "user", "content": prompt_1.format(problem=problem)}]

    response_1 = get_groq_response(chat_history)
    # print(f"Response 1: {response_1}")

    initial_answer = get_groq_response([{"role": "user", "content": response_1+"\n"+extraction_prompt}])

    chat_history.append({"role": "assistant", "content": response_1})
    chat_history.append({"role": "user", "content": prompt_2})

    # Prompt 2: Review the previous answer
    response_2 = get_groq_response(chat_history)
    # print(f"Response 2: {response_2}")

    chat_history.append({"role": "assistant", "content": response_2})
    chat_history.append({"role": "user", "content": prompt_3})

    # Prompt 3: Improve the answer based on the review
    response_3 = get_groq_response(chat_history)

    final_answer = get_groq_response([{"role": "user", "content": response_3+"\n"+extraction_prompt}])
    # print(f"Response 3: {response_3}")

    return response_1, response_2, response_3, initial_answer, final_answer

doubtful_responses = []

def extract_answer(response):

    start_index = response.find("\\boxed{") + len("\\boxed{")

    end_index = response.find("}", start_index)

    answer = response[start_index:end_index]

    # if(end_index == -1):
    #     end_index = response.find("]", start_index)
    # if(end_index == -1):
    #     end_index = response.find(")", start_index)

    # print("Response: ", response)

    # print("Start: ", start_index)
    # print("End: ", end_index)

    # print("Value: "+response[start_index:end_index])

    # answer = ""
    # flag = False
    # for i in range(start_index, end_index):
    #     if response[i].isdigit():
    #         flag = True
    #         answer += response[i]
    #     else:
    #         if flag:
    #             flag=False
    #             break

    return int(answer)

# Evaluate on the first 5 problems in the dataset

correct_1 = 0
correct_2 = 0

for idx in range(len(dataset)):
    print("Idx: ", idx)
    problem = dataset[idx]['Body'] + " " + dataset[idx]['Question']
    correct_answer = int(dataset[idx]['Answer'])
    # print(f"\nProblem {idx+1}: {problem}")
    response_1, response_2, response_3, initial_answer, final_answer = evaluate_self_correction(problem)

    # print("Response 1: ", response_1)
    # print("Response 2: ", response_2)
    # print("Response 3: ", response_3)

    try:
        initial_answer = extract_answer(initial_answer)
    except:
        doubtful_responses.append(response_1)
        continue
    # try:
    #     answer_1 = int(answer_1.split(" ")[0])
    # except: 
    #     print("Response 1: "+ response_1)
    #     print("Answer 1: "+ answer_1)
    #     break
    # answer_2 = extract_answer(response_2)
    try:
        final_answer = extract_answer(final_answer)
    except: 
        doubtful_responses.append(response_3)
        continue
    # try:
    #     answer_2 = int(answer_2.split(" ")[0])
    # except: 
    #     print("Response 2: "+ response_3)
    #     print("Answer 2: "+ answer_2)
    #     break
    # print("Initial Answer: ", initial_answer)
    # print("Final Answer: ", final_answer)
    
    if(initial_answer == correct_answer):
        correct_1 += 1
    if(final_answer == correct_answer):
        correct_2 += 1


accuracy_1 = correct_1/(300-len(doubtful_responses)) * 100
accuracy_2 = correct_2/(300-len(doubtful_responses)) * 100

print("Accuracy before self-correction: ", accuracy_1)
print("Accuracy after self-correction: ", accuracy_2)

print("Incorrect Format: ", len(doubtful_responses))






Idx:  0
Idx:  1
Idx:  2
Idx:  3
Idx:  4
Idx:  5
Idx:  6
Idx:  7
Idx:  8
Idx:  9
Idx:  10
Idx:  11
Idx:  12
Idx:  13
Idx:  14
Idx:  15
Idx:  16
Idx:  17
Idx:  18
Idx:  19
Idx:  20
Idx:  21
Idx:  22
Idx:  23
Idx:  24
Idx:  25
Idx:  26
Idx:  27
Idx:  28
Idx:  29
Idx:  30
Idx:  31
Idx:  32
Idx:  33
Idx:  34
Idx:  35
Idx:  36
Idx:  37
Idx:  38
Idx:  39
Idx:  40
Idx:  41
Idx:  42
Idx:  43
Idx:  44
Idx:  45
Idx:  46
Idx:  47
Idx:  48
Idx:  49
Idx:  50
Idx:  51
Idx:  52
Idx:  53
Idx:  54
Idx:  55
Idx:  56
Idx:  57
Idx:  58
Idx:  59
Idx:  60
Idx:  61
Idx:  62
Idx:  63
Idx:  64
Idx:  65
Idx:  66
Idx:  67
Idx:  68
Idx:  69
Idx:  70
Idx:  71
Idx:  72
Idx:  73
Idx:  74
Idx:  75
Idx:  76
Idx:  77
Idx:  78
Idx:  79
Idx:  80
Idx:  81
Idx:  82
Idx:  83
Idx:  84
Idx:  85
Idx:  86


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01j8wk53dnejab3ymjjsy4vnwk` on : Limit 500000, Used 498518, Requested 2123. Please try again in 1m50.7404s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

21